In [3]:
# Dependencies and Setup
import hvplot.pandas
import holoviews


# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import datetime

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [8]:
hap_df = pd.read_csv('./Resources/2022.csv')

cost_df = pd.read_csv('./Resources/Cost_of_Living_Index_2022.csv')

print(hap_df.head())
print(cost_df.head())

   RANK      Country Happiness score Whisker-high Whisker-low  \
0     1      Finland           7,821        7,886       7,756   
1     2      Denmark           7,636        7,710       7,563   
2     3      Iceland           7,557        7,651       7,464   
3     4  Switzerland           7,512        7,586       7,437   
4     5  Netherlands           7,415        7,471       7,359   

  Dystopia (1.83) + residual Explained by: GDP per capita  \
0                      2,518                        1,892   
1                      2,226                        1,953   
2                      2,320                        1,936   
3                      2,153                        2,026   
4                      2,137                        1,945   

  Explained by: Social support Explained by: Healthy life expectancy  \
0                        1,258                                 0,775   
1                        1,243                                 0,777   
2                        1

In [9]:
# Merging
mHapCost_df = hap_df.merge(cost_df, how='inner', on='Country')
mHapCost_df.head()

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,Finland,"7,821","7,886","7,756","2,518","1,892","1,258","0,775","0,736","0,109","0,534",41,73.20,25.95,51.05,65.16,77.65,91.02
1,2,Denmark,"7,636","7,710","7,563","2,226","1,953","1,243","0,777","0,719","0,188","0,532",33,84.12,33.23,60.26,68.60,98.75,99.45
2,3,Iceland,"7,557","7,651","7,464","2,320","1,936","1,320","0,803","0,718","0,270","0,191",52,94.86,41.93,70.05,90.22,99.42,77.06
3,4,Switzerland,"7,512","7,586","7,437","2,153","2,026","1,226","0,822","0,677","0,147","0,461",120,123.35,53.54,90.62,128.13,122.09,118.44
4,5,Netherlands,"7,415","7,471","7,359","2,137","1,945","1,206","0,787","0,651","0,271","0,419",86,75.66,36.06,57.10,65.49,76.35,87.99


In [10]:
mHapCost_df.count()

RANK                                          106
Country                                       106
Happiness score                               106
Whisker-high                                  106
Whisker-low                                   106
Dystopia (1.83) + residual                    106
Explained by: GDP per capita                  106
Explained by: Social support                  106
Explained by: Healthy life expectancy         106
Explained by: Freedom to make life choices    106
Explained by: Generosity                      106
Explained by: Perceptions of corruption       106
Rank                                          106
Cost of Living Index                          106
Rent Index                                    106
Cost of Living Plus Rent Index                106
Groceries Index                               106
Restaurant Price Index                        106
Local Purchasing Power Index                  106
dtype: int64

In [47]:
crime_df = pd.read_csv('./Resources/WorldCrimeIndex.csv')
crime_df.head()

,Rank,City,Crime Index,Safety Index
0,1,"Caracas, Venezuela",83.98,16.02
1,2,"Pretoria, South Africa",81.98,18.02
2,3,"Celaya, Mexico",81.80,18.20
3,4,"San Pedro Sula, Honduras",80.87,19.13
4,5,"Port Moresby, Papua New Guinea",80.71,19.29


In [48]:
df = crime_df['City'].str.split(",", expand=True)
# crime_df.rename(columns={'City':"Country"})
df.head()

,0,1,2
0,Caracas,Venezuela,None
1,Pretoria,South Africa,None
2,Celaya,Mexico,None
3,San Pedro Sula,Honduras,None
4,Port Moresby,Papua New Guinea,None


In [49]:
crime_df['City'] = df[1]
crime_df.head()

,Rank,City,Crime Index,Safety Index
0,1,Venezuela,83.98,16.02
1,2,South Africa,81.98,18.02
2,3,Mexico,81.80,18.20
3,4,Honduras,80.87,19.13
4,5,Papua New Guinea,80.71,19.29


In [54]:

crime_df.columns=['Crime Rank', 'Country', 'Crime Index', 'Safety Index']
print(crime_df.count())
sr_df = crime_df.sort_values('Country')
print(sr_df.head(20))
crime_df.head()

Crime Rank      453
Country         453
Crime Index     453
Safety Index    453
dtype: int64
     Crime Rank       Country  Crime Index  Safety Index
54           55            AB        63.89         36.11
67           68            AK        62.27         37.73
165         166            AZ        50.29         49.71
150         151            AZ        52.23         47.77
7             8   Afghanistan        79.39         20.61
229         230       Albania        44.50         55.50
129         130       Algeria        53.88         46.12
39           40        Angola        67.45         32.55
76           77     Argentina        60.38         39.62
16           17     Argentina        75.11         24.89
60           61     Argentina        63.33         36.67
427         428       Armenia        21.66         78.34
105         106     Australia        55.76         44.24
75           76     Australia        60.57         39.43
318         319     Australia        35.05         6

,Crime Rank,Country,Crime Index,Safety Index
0,1,Venezuela,83.98,16.02
1,2,South Africa,81.98,18.02
2,3,Mexico,81.80,18.20
3,4,Honduras,80.87,19.13
4,5,Papua New Guinea,80.71,19.29


In [51]:
mHapCosCrim = mHapCost_df.merge(crime_df, how='inner', on='Country')
mHapCosCrim.head()

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,...,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Crime Rank,Crime Index,Safety Index
